In [ ]:
'''
data loading and preprocess
1. data loading
2. calculate max values and mean values
3. data normalize
4. 

'''

In [1]:
import torch
import numpy as np
import pandas as pd
import os
import math
import warnings
import itertools
import numbers
import torch.utils.data as utils

In [2]:
inputpath = './input/set-a/'
inputdict = {
    "ALP" : 0,
    "ALT" : 1,
    "AST" : 2,
    "Albumin" : 3,
    "BUN" : 4,
    "Bilirubin" : 5,
    "Cholesterol" : 6,
    "Creatinine" : 7,
    "DiasABP" : 8,
    "FiO2" : 9,
    "GCS" : 10,
    "Glucose" : 11,
    "HCO3" : 12,
    "HCT" : 13,
    "HR" : 14,
    "K" : 15,
    "Lactate" : 16,
    "MAP" : 17,
    "Mg" : 18,
    "Na" : 19,
    "PaCO2" : 20,
    "PaO2" : 21,
    "Platelets" : 22,
    "RespRate" : 23,
    "SaO2" : 24,
    "SysABP" : 25,
    "Temp" : 26,
    "Tropl" : 27,
    "TroponinI" : 27, #temp
    "TropT" : 28,
    "TroponinT" : 28, #temp
    "Urine" : 29,
    "WBC" : 30,
    "Weight" : 31,
    "pH" : 32,
    "NIDiasABP" : 33,
    "NIMAP" : 34,
    "NISysABP" : 35,
    "MechVent" : 36,
    "RecordID" : 37,
    "Age" : 38,
    "Gender" :39,
    "ICUType" : 40,
    "Height": 41
}

In [3]:
def timeparser(time):
    return pd.to_timedelta(time + ':00')

def timedelta_to_day_figure(timedelta):
    return timedelta.days + (timedelta.seconds/86400) #(24*60*60)

In [4]:
def df_to_x(df):
    grouped_data = df.groupby('Time')
    
    #generate input vectors
    x = np.zeros((len(inputdict)-2, grouped_data.ngroups))

    # fill the x and masking vectors
    pre_time = pd.to_timedelta(0)
    t = 0
    for row_index, value in df.iterrows():
        '''
        t = colum, time frame
        agg_no = row, variable
        '''
        #print(value)
        agg_no = inputdict[value.Parameter]

        # same timeline check.        
        if pre_time != value.Time:
            pre_time = value.Time
            t += 1
            
        #print('agg_no : {}\t t : {}\t value : {}'.format(agg_no, t, value.Value))
        x[agg_no, t] = value.Value    
     
    return x

In [5]:
x_max = np.zeros(len(inputdict) -2)
for filename in os.listdir(inputpath):
    df = pd.read_csv(inputpath + filename,\
                     header=0,\
                     parse_dates=['Time'],\
                     date_parser=timeparser)
    x = df_to_x(df)
    temp_x_max = np.max(x.T, axis=0)
    x_max = np.maximum(x_max, temp_x_max)
print(x_max)

[2.20500e+03 1.14700e+04 1.84300e+04 5.30000e+00 1.97000e+02 4.77000e+01
 3.30000e+02 2.21000e+01 2.68000e+02 1.00000e+00 1.50000e+01 1.14300e+03
 5.00000e+01 6.18000e+01 3.00000e+02 2.29000e+01 2.93000e+01 3.00000e+02
 9.90000e+00 1.77000e+02 1.00000e+02 5.00000e+02 1.04700e+03 9.80000e+01
 1.00000e+02 2.95000e+02 4.21000e+01 4.92000e+01 2.49100e+01 1.10000e+04
 1.87500e+02 3.00000e+02 7.35000e+02 2.01000e+02 2.09000e+02 2.96000e+02
 1.00000e+00 1.42673e+05 9.00000e+01 1.00000e+00 4.00000e+00 4.31800e+02]


In [6]:
slice_x_max = x_max[:33]
print(slice_x_max)
print(slice_x_max.shape)

[2.205e+03 1.147e+04 1.843e+04 5.300e+00 1.970e+02 4.770e+01 3.300e+02
 2.210e+01 2.680e+02 1.000e+00 1.500e+01 1.143e+03 5.000e+01 6.180e+01
 3.000e+02 2.290e+01 2.930e+01 3.000e+02 9.900e+00 1.770e+02 1.000e+02
 5.000e+02 1.047e+03 9.800e+01 1.000e+02 2.950e+02 4.210e+01 4.920e+01
 2.491e+01 1.100e+04 1.875e+02 3.000e+02 7.350e+02]
(33,)


In [7]:
np.save('./input/x_max', slice_x_max)

l_x_max = np.load('./input/x_max.npy')
print(l_x_max.shape)
print(l_x_max)

(33,)
[2.205e+03 1.147e+04 1.843e+04 5.300e+00 1.970e+02 4.770e+01 3.300e+02
 2.210e+01 2.680e+02 1.000e+00 1.500e+01 1.143e+03 5.000e+01 6.180e+01
 3.000e+02 2.290e+01 2.930e+01 3.000e+02 9.900e+00 1.770e+02 1.000e+02
 5.000e+02 1.047e+03 9.800e+01 1.000e+02 2.950e+02 4.210e+01 4.920e+01
 2.491e+01 1.100e+04 1.875e+02 3.000e+02 7.350e+02]


In [ ]:
### mean values

In [8]:
def df_to_sum_x(df, x_sum):
    for row_index, value in df.iterrows():
        '''
        0 : values
        1 : # of values observed
        agg_no = row, variable
        '''
        
        #print(value)
        agg_no = inputdict[value.Parameter]
        
        x_sum[agg_no, 0] += value.Value
        x_sum[agg_no, 1] += 1 

    return x_sum

In [9]:
#generate input vectors
x_sum = np.zeros((len(inputdict)-2, 2))

for filename in os.listdir(inputpath):
    df = pd.read_csv(inputpath + filename,\
                     header=0,\
                     parse_dates=['Time'],\
                     date_parser=timeparser)
    x_sum = df_to_sum_x(df, x_sum)
print(x_sum.shape)

(42, 2)


In [10]:
x_mean = x_sum[:,:1]/x_sum[:,1:2]

'''print(x_sum[:,:1])
print(x_sum[:,1:2])

print(x_mean.shape)
print(x_mean)'''
print(x_mean.shape)

(42, 1)


In [11]:
x_mean = np.squeeze(x_mean)
print(x_mean.shape)
#print(x_mean)
x_mean = x_mean[:33]
print(x_mean.shape)
#print(x_mean)

(42,)
(33,)


In [12]:
np.save('./input/x_mean', x_mean)
l_x_mean = np.load('./input/x_mean.npy')
print(l_x_mean.shape)
#print(l_x_mean)

(33,)


In [13]:
x_max = np.load('./input/x_max.npy')
x_nor_mean = l_x_mean / x_max
print(x_nor_mean.shape)
print(x_nor_mean)

(33,)
[0.05295841 0.03439189 0.02747065 0.55138226 0.13917019 0.060976
 0.47429533 0.06809853 0.22123325 0.54520145 0.75991423 0.12380539
 0.46231485 0.49642465 0.29172481 0.18062091 0.09982182 0.26580976
 0.20483942 0.78573915 0.40474748 0.30083054 0.18224997 0.20126254
 0.96637068 0.40236576 0.8790416  0.1453509  0.04811124 0.0109817
 0.06757378 0.27795538 0.01018939]


In [14]:
np.save('./input/x_nor_mean', x_nor_mean)
 
l_x_nor_mean = np.load('./input/x_nor_mean.npy')
print(l_x_nor_mean.shape)
print(l_x_nor_mean)

(33,)
[0.05295841 0.03439189 0.02747065 0.55138226 0.13917019 0.060976
 0.47429533 0.06809853 0.22123325 0.54520145 0.75991423 0.12380539
 0.46231485 0.49642465 0.29172481 0.18062091 0.09982182 0.26580976
 0.20483942 0.78573915 0.40474748 0.30083054 0.18224997 0.20126254
 0.96637068 0.40236576 0.8790416  0.1453509  0.04811124 0.0109817
 0.06757378 0.27795538 0.01018939]


In [15]:
def df_to_x_m_d(df, inputdic, size, id_posistion, split, max_input):
    grouped_data = df.groupby('Time')
    
    #generate input vectors
    x = np.zeros((len(inputdic)-2, grouped_data.ngroups))
    masking = np.zeros((len(inputdic)-2, grouped_data.ngroups))
    delta = np.zeros((split, size))
    timetable = np.zeros(grouped_data.ngroups)
    id = 0
    
    s_dataset = np.zeros((3, split, size))
    
    if grouped_data.ngroups > size:
        
        # fill the x and masking vectors
        pre_time = pd.to_timedelta(0)
        t = 0
        for row_index, value in df.iterrows():
            '''
            t = colum, time frame
            agg_no = row, variable
            '''
            #print(value)
            agg_no = inputdict[value.Parameter]

            # same timeline check.        
            if pre_time != value.Time:
                pre_time = value.Time
                t += 1
                timetable[t] = timedelta_to_day_figure(value.Time)

            #print('agg_no : {}\t t : {}\t value : {}'.format(agg_no, t, value.Value))
            x[agg_no, t] = value.Value    
            masking[agg_no, t] = 1
        
        # generate random index array 
        ran_index = np.random.choice(grouped_data.ngroups, size=size, replace=False)
        ran_index.sort()
        ran_index[0] = 0
        ran_index[size-1] = grouped_data.ngroups-1
        
        # take id for outcome comparing
        id = x[id_posistion, 0]
        
        # remove unnesserly parts(rows)
        x = x[:split, :]
        masking = masking[:split, :]
        
        # coulme(time) sampling
        x_sample = np.zeros((split, size))
        m_sample = np.zeros((split, size))
        time_sample = np.zeros(size)

        t_x_sample = x_sample.T
        t_marsking = m_sample.T
        #t_time = t_sample.T
        
        t_x = x.T
        t_m = masking.T
        #t_t = t.T

        it = np.nditer(ran_index, flags=['f_index'])
        while not it.finished:
            #print('it.index = {}, it[0] = {}, ran_index = {}'.format(it.index, it[0], ran_index[it.index]))
            t_x_sample[it.index] = t_x[it[0]]
            t_marsking[it.index] = t_m[it[0]]
            time_sample[it.index] = timetable[it[0]]
            it.iternext()
        
        x = x_sample
        masking = m_sample
        timetable = time_sample
        
        # normalize the X
        nor_x = x/max_input[:, np.newaxis]
     
        # fill the delta vectors
        for index, value in np.ndenumerate(masking):
            '''
            index[0] = row, agg
            index[1] = col, time
            '''
            if index[1] == 0:
                delta[index[0], index[1]] = 0
            elif masking[index[0], index[1]-1] == 0:
                delta[index[0], index[1]] = timetable[index[1]] - timetable[index[1]-1] + delta[index[0], index[1]-1]
            else:
                delta[index[0], index[1]] = timetable[index[1]] - timetable[index[1]-1]
    
    else:
                
        # fill the x and masking vectors
        pre_time = pd.to_timedelta(0)
        t = 0
        for row_index, value in df.iterrows():
            '''
            t = colum, time frame
            agg_no = row, variable
            '''
            #print(value)
            agg_no = inputdict[value.Parameter]

            # same timeline check.        
            if pre_time != value.Time:
                pre_time = value.Time
                t += 1
                timetable[t] = timedelta_to_day_figure(value.Time)

            #print('agg_no : {}\t t : {}\t value : {}'.format(agg_no, t, value.Value))
            x[agg_no, t] = value.Value    
            masking[agg_no, t] = 1
        
        # take id for outcome comparing
        id = x[id_posistion, 0]
        
        # remove unnesserly parts(rows)
        x = x[:split, :]
        masking = masking[:split, :]
        
        x = np.pad(x, ((0,0), (size-grouped_data.ngroups, 0)), 'constant')
        masking = np.pad(masking, ((0,0), (size-grouped_data.ngroups, 0)), 'constant')
        timetable = np.pad(timetable, (size-grouped_data.ngroups, 0), 'constant')
        
        # normalize the X
        nor_x = x/max_input[:, np.newaxis]
        
        # fill the delta vectors
        for index, value in np.ndenumerate(masking):
            '''
            index[0] = row, agg
            index[1] = col, time
            '''
            if index[1] == 0:
                delta[index[0], index[1]] = 0
            elif masking[index[0], index[1]-1] == 0:
                delta[index[0], index[1]] = timetable[index[1]] - timetable[index[1]-1] + delta[index[0], index[1]-1]
            else:
                delta[index[0], index[1]] = timetable[index[1]] - timetable[index[1]-1]
                
    s_dataset[0] = x
    s_dataset[1] = masking
    s_dataset[2] = delta
    
    return s_dataset, id

In [16]:
# def df_to_x_m_d(df, inputdic, size, id_posistion, split, max_input):
size = 49
id_posistion = 37
split = 33
dataset = np.zeros((1,3, split, size))
max_input = np.load('./input/x_max.npy')

for filename in os.listdir(inputpath):
    df = pd.read_csv(inputpath + filename,\
                     header=0,\
                     parse_dates=['Time'],\
                     date_parser=timeparser)
    s_dataset, id = df_to_x_m_d(df, inputdict, size, id_posistion, split, max_input)
    
    dataset = np.concatenate((dataset, s_dataset[np.newaxis, :,:,:]))
    

dataset = dataset[1:, :,:,:]    
print(dataset.shape)

print(dataset[0].shape)

print(dataset[0][0])

(4000, 3, 33, 49)
(3, 33, 49)
[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [-1.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [17]:
np.save('./input/dataset', dataset)

t_dataset = np.load('./input/dataset.npy')

print(t_dataset.shape)

(4000, 3, 33, 49)


In [37]:
### Y or label data
A_outcome = pd.read_csv('./input/Outcomes-a.txt')
arr_outcome = A_outcome.values

out_max = np.amax(arr_outcome, axis=0)
print(out_max)

[142673     34     22    154   2600      1]


In [38]:
nor_outcomes = arr_outcome/out_max[np.newaxis, :]
s_nor_outcomes = nor_outcomes[:,1:]
print(s_nor_outcomes)

s_out_max = out_max[1:]
print(s_out_max)

[[ 1.76470588e-01  4.54545455e-02  3.24675325e-02 -3.84615385e-04
   0.00000000e+00]
 [ 4.70588235e-01  3.63636364e-01  5.19480519e-02 -3.84615385e-04
   0.00000000e+00]
 [ 6.17647059e-01  5.00000000e-01  1.23376623e-01 -3.84615385e-04
   0.00000000e+00]
 ...
 [ 2.35294118e-01  2.27272727e-01  7.14285714e-02 -3.84615385e-04
   0.00000000e+00]
 [ 6.47058824e-01  4.54545455e-01  5.19480519e-02  2.69230769e-03
   1.00000000e+00]
 [ 7.35294118e-01  5.00000000e-01  4.54545455e-02 -3.84615385e-04
   0.00000000e+00]]
[  34   22  154 2600    1]


In [39]:
s_nor_outcomes = nor_outcomes[:,1:]
print(s_nor_outcomes)

s_out_max = out_max[1:]
print(s_out_max)

[[ 1.76470588e-01  4.54545455e-02  3.24675325e-02 -3.84615385e-04
   0.00000000e+00]
 [ 4.70588235e-01  3.63636364e-01  5.19480519e-02 -3.84615385e-04
   0.00000000e+00]
 [ 6.17647059e-01  5.00000000e-01  1.23376623e-01 -3.84615385e-04
   0.00000000e+00]
 ...
 [ 2.35294118e-01  2.27272727e-01  7.14285714e-02 -3.84615385e-04
   0.00000000e+00]
 [ 6.47058824e-01  4.54545455e-01  5.19480519e-02  2.69230769e-03
   1.00000000e+00]
 [ 7.35294118e-01  5.00000000e-01  4.54545455e-02 -3.84615385e-04
   0.00000000e+00]]
[  34   22  154 2600    1]


In [40]:
np.save('./input/max_out', s_out_max)
np.save('./input/nor_out', s_nor_outcomes)

l_max = np.load('./input/max_out.npy')
l_out = np.load('./input/nor_out.npy')

print(l_max.shape)
print(l_out.shape)

(5,)
(4000, 5)


In [41]:
class GRUD(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, x_mean=0,\
                 bias=True, batch_first=False, dropout=0, bidirectional=False):
        super(GRUD, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.zeros = torch.autograd.Variable(torch.zeros(input_size))
        self.x_mean = torch.autograd.Variable(torch.tensor(x_mean))
        self.bias = bias
        self.batch_first = batch_first
        self.dropout = dropout
        self.bidirectional = bidirectional
        num_directions = 2 if bidirectional else 1
        
        if not isinstance(dropout, numbers.Number) or not 0 <= dropout <= 1 or \
                isinstance(dropout, bool):
            raise ValueError("dropout should be a number in range [0, 1] "
                             "representing the probability of an element being "
                             "zeroed")
        if dropout > 0 and num_layers == 1:
            warnings.warn("dropout option adds dropout after all but last "
                          "recurrent layer, so non-zero dropout expects "
                          "num_layers greater than 1, but got dropout={} and "
                          "num_layers={}".format(dropout, num_layers))
        
        ################################
        gate_size = 1 # not used
        ################################
        
        self._all_weights = []
        for layer in range(num_layers):
            for direction in range(num_directions):
                # not used
                layer_input_size = input_size if layer == 0 else hidden_size * num_directions
             
                # decay rates gamma
                w_dg_x = torch.nn.Parameter(torch.Tensor(input_size))
                w_dg_h = torch.nn.Parameter(torch.Tensor(hidden_size))
                                
                # z
                w_xz = torch.nn.Parameter(torch.Tensor(input_size))
                w_hz = torch.nn.Parameter(torch.Tensor(hidden_size))
                w_mz = torch.nn.Parameter(torch.Tensor(input_size))
                
                # r
                w_xr = torch.nn.Parameter(torch.Tensor(input_size))
                w_hr = torch.nn.Parameter(torch.Tensor(hidden_size))
                w_mr = torch.nn.Parameter(torch.Tensor(input_size))
                
                # h_tilde
                w_xh = torch.nn.Parameter(torch.Tensor(input_size))
                w_hh = torch.nn.Parameter(torch.Tensor(hidden_size))
                w_mh = torch.nn.Parameter(torch.Tensor(input_size))
                
                # y (output)
                w_hy = torch.nn.Parameter(torch.Tensor(output_size, hidden_size))
                
                # bias
                b_dg_x = torch.nn.Parameter(torch.Tensor(hidden_size))
                b_dg_h = torch.nn.Parameter(torch.Tensor(hidden_size))
                b_z = torch.nn.Parameter(torch.Tensor(hidden_size))
                b_r = torch.nn.Parameter(torch.Tensor(hidden_size))
                b_h = torch.nn.Parameter(torch.Tensor(hidden_size))
                b_y = torch.nn.Parameter(torch.Tensor(output_size))
                
                layer_params = (w_dg_x, w_dg_h,\
                                w_xz, w_hz, w_mz,\
                                w_xr, w_hr, w_mr,\
                                w_xh, w_hh, w_mh,\
                                w_hy,\
                                b_dg_x, b_dg_h, b_z, b_r, b_h, b_y)
                
                suffix = '_reverse' if direction == 1 else ''
                param_names = ['weight_dg_x_l{}{}', 'weight_dg_h_l{}{}',\
                               'weight_xz_l{}{}', 'weight_hz_l{}{}','weight_mz_l{}{}',\
                               'weight_xr_l{}{}', 'weight_hr_l{}{}','weight_mr_l{}{}',\
                               'weight_xh_l{}{}', 'weight_hh_l{}{}','weight_mh_l{}{}',\
                               'weight_hy']
                if bias:
                    param_names += ['bias_dg_x_l{}{}', 'bias_dg_h_l{}{}',\
                                    'bias_z_l{}{}',\
                                    'bias_r_l{}{}',\
                                    'bias_h_l{}{}',\
                                    'bias_y']
                param_names = [x.format(layer, suffix) for x in param_names]

                for name, param in zip(param_names, layer_params):
                    setattr(self, name, param)
                self._all_weights.append(param_names)
        
        self.flatten_parameters()
        self.reset_parameters()
        
    def flatten_parameters(self):
        """
        Resets parameter data pointer so that they can use faster code paths.
        Right now, this works only if the module is on the GPU and cuDNN is enabled.
        Otherwise, it's a no-op.
        """
        any_param = next(self.parameters()).data
        if not any_param.is_cuda or not torch.backends.cudnn.is_acceptable(any_param):
            return

        # If any parameters alias, we fall back to the slower, copying code path. This is
        # a sufficient check, because overlapping parameter buffers that don't completely
        # alias would break the assumptions of the uniqueness check in
        # Module.named_parameters().
        all_weights = self._flat_weights
        unique_data_ptrs = set(p.data_ptr() for p in all_weights)
        if len(unique_data_ptrs) != len(all_weights):
            return

        with torch.cuda.device_of(any_param):
            import torch.backends.cudnn.rnn as rnn

            # NB: This is a temporary hack while we still don't have Tensor
            # bindings for ATen functions
            with torch.no_grad():
                # NB: this is an INPLACE function on all_weights, that's why the
                # no_grad() is necessary.
                torch._cudnn_rnn_flatten_weight(
                    all_weights, (4 if self.bias else 2),
                    self.input_size, rnn.get_cudnn_mode(self.mode), self.hidden_size, self.num_layers,
                    self.batch_first, bool(self.bidirectional))

    def _apply(self, fn):
        ret = super(GRUD, self)._apply(fn)
        self.flatten_parameters()
        return ret

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            torch.nn.init.uniform_(weight, -stdv, stdv)

    def check_forward_args(self, input, hidden, batch_sizes):
        is_input_packed = batch_sizes is not None
        expected_input_dim = 2 if is_input_packed else 3
        if input.dim() != expected_input_dim:
            raise RuntimeError(
                'input must have {} dimensions, got {}'.format(
                    expected_input_dim, input.dim()))
        if self.input_size != input.size(-1):
            raise RuntimeError(
                'input.size(-1) must be equal to input_size. Expected {}, got {}'.format(
                    self.input_size, input.size(-1)))

        if is_input_packed:
            mini_batch = int(batch_sizes[0])
        else:
            mini_batch = input.size(0) if self.batch_first else input.size(1)

        num_directions = 2 if self.bidirectional else 1
        expected_hidden_size = (self.num_layers * num_directions,
                                mini_batch, self.hidden_size)
        
        def check_hidden_size(hx, expected_hidden_size, msg='Expected hidden size {}, got {}'):
            if tuple(hx.size()) != expected_hidden_size:
                raise RuntimeError(msg.format(expected_hidden_size, tuple(hx.size())))

        if self.mode == 'LSTM':
            check_hidden_size(hidden[0], expected_hidden_size,
                              'Expected hidden[0] size {}, got {}')
            check_hidden_size(hidden[1], expected_hidden_size,
                              'Expected hidden[1] size {}, got {}')
        else:
            check_hidden_size(hidden, expected_hidden_size)
    
    def extra_repr(self):
        s = '{input_size}, {hidden_size}'
        if self.num_layers != 1:
            s += ', num_layers={num_layers}'
        if self.bias is not True:
            s += ', bias={bias}'
        if self.batch_first is not False:
            s += ', batch_first={batch_first}'
        if self.dropout != 0:
            s += ', dropout={dropout}'
        if self.bidirectional is not False:
            s += ', bidirectional={bidirectional}'
        return s.format(**self.__dict__)
    
    
    def __setstate__(self, d):
        super(GRUD, self).__setstate__(d)
        if 'all_weights' in d:
            self._all_weights = d['all_weights']
        if isinstance(self._all_weights[0][0], str):
            return
        num_layers = self.num_layers
        num_directions = 2 if self.bidirectional else 1
        self._all_weights = []
        for layer in range(num_layers):
            for direction in range(num_directions):
                suffix = '_reverse' if direction == 1 else ''
                weights = ['weight_dg_x_l{}{}', 'weight_dg_h_l{}{}',\
                           'weight_xz_l{}{}', 'weight_hz_l{}{}','weight_mz_l{}{}',\
                           'weight_xr_l{}{}', 'weight_hr_l{}{}','weight_mr_l{}{}',\
                           'weight_xh_l{}{}', 'weight_hh_l{}{}','weight_mh_l{}{}',\
                           'weight_hy',\
                           'bias_dg_x_l{}{}', 'bias_dg_h_l{}{}',\
                           'bias_z_l{}{}', 'bias_r_l{}{}', 'bias_h_l{}{}','bias_y']
                weights = [x.format(layer, suffix) for x in weights]
                if self.bias:
                    self._all_weights += [weights]
                else:
                    self._all_weights += [weights[:2]]

    @property
    def _flat_weights(self):
        return list(self._parameters.values())

    @property
    def all_weights(self):
        return [[getattr(self, weight) for weight in weights] for weights in self._all_weights]
    
    def forward(self, input):
        # input.size = (3, 33,49) : num_input or num_hidden, num_layer or step
        X = torch.squeeze(input[0]) # .size = (33,49)
        Mask = torch.squeeze(input[1]) # .size = (33,49)
        Delta = torch.squeeze(input[2]) # .size = (33,49)
        Hidden_State = torch.autograd.Variable(torch.zeros(input_size))
        
        step_size = X.size(1) # 49
        #print('step size : ', step_size)
        
        output = None
        h = Hidden_State
        for layer in range(num_layers):
            
            x = torch.squeeze(X[:,layer:layer+1])
            m = torch.squeeze(Mask[:,layer:layer+1])
            d = torch.squeeze(Delta[:,layer:layer+1])
            
            # decay rates gamma
            w_dg_x = getattr(self, 'weight_dg_x_l' + str(layer))
            w_dg_h = getattr(self, 'weight_dg_h_l' + str(layer))
                                
            #z
            w_xz = getattr(self, 'weight_xz_l' + str(layer))
            w_hz = getattr(self, 'weight_hz_l' + str(layer))
            w_mz = getattr(self, 'weight_mz_l' + str(layer))
                
            # r
            w_xr = getattr(self, 'weight_xr_l' + str(layer))
            w_hr = getattr(self, 'weight_hr_l' + str(layer))
            w_mr = getattr(self, 'weight_mr_l' + str(layer))
                
            # h_tilde
            w_xh = getattr(self, 'weight_xh_l' + str(layer))
            w_hh = getattr(self, 'weight_hh_l' + str(layer))
            w_mh = getattr(self, 'weight_mh_l' + str(layer))
                
            # bias
            b_dg_x = getattr(self, 'bias_dg_x_l' + str(layer))
            b_dg_h = getattr(self, 'bias_dg_h_l' + str(layer))
            b_z = getattr(self, 'bias_z_l' + str(layer))
            b_r = getattr(self, 'bias_r_l' + str(layer))
            b_h = getattr(self, 'bias_h_l' + str(layer))
            
            #(4)
            gamma_x = torch.exp(-torch.max(self.zeros, (w_dg_x * x + b_dg_x)))
            gamma_h = torch.exp(-torch.max(self.zeros, (w_dg_h * h + b_dg_h)))

            #(5)
            x = m * x + (1 - m) * (gamma_x * x + (1 - gamma_x) * self.x_mean)

            #(6)
            h = gamma_h * h

            z = torch.nn.functional.sigmoid((w_xz*x + w_hz*h + w_mz*m + b_z))
            r = torch.nn.functional.sigmoid((w_xr*x + w_hr*h + w_mr*m + b_r))
            h_tilde = torch.nn.functional.tanh((w_xh*x + w_hh*(gamma_h * h) + w_mh*m + b_h))
            h = (1 - z) * h + z * h_tilde 
            
        w_hy = getattr(self, 'weight_hy')
        b_y = getattr(self, 'bias_y')

        output = torch.matmul(w_hy, h) + b_y
        
        return output

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [21]:
def data_dataloader(dataset, outcomes,\
                    train_proportion = 0.7,\
                    dev_proportion = 0.15):
    
    train_index = int(np.floor(dataset.shape[0] * train_proportion))
    dev_index = int(np.floor(dataset.shape[0] * (train_proportion + dev_proportion)))
    
    print('train_index : {}'.format(train_index))
    print('dev_index : {}'.format(dev_index))
    
    # split dataset to tarin/dev/test set
    train_data, train_label = dataset[:train_index, :,:,:], outcomes[:train_index, :]
    dev_data, dev_label = dataset[train_index:dev_index, :,:,:], outcomes[train_index:dev_index, :]
    test_data, test_label = dataset[dev_index:, :,:,:], outcomes[dev_index:, :]
    
    print("train_data.shape : {}".format(train_data.shape))
    print(train_label.shape)
    print(dev_data.shape)
    print(dev_label.shape)
    print(test_data.shape)
    print(test_label.shape)    
    
    # ndarray to tensor
    train_data, train_label = torch.Tensor(train_data), torch.Tensor(train_label)
    dev_data, dev_label = torch.Tensor(dev_data), torch.Tensor(dev_label)
    test_data, test_label = torch.Tensor(test_data), torch.Tensor(test_label)
    
    print(test_data.shape)
    print(test_label.shape)
    
    # tensor to dataset
    train_dataset = utils.TensorDataset(train_data, train_label)
    dev_dataset = utils.TensorDataset(dev_data, dev_label)
    test_dataset = utils.TensorDataset(test_data, test_label)
    
    # dataset to dataloader 
    train_dataloader = utils.DataLoader(train_dataset)
    dev_dataloader = utils.DataLoader(dev_dataset)
    test_dataloader = utils.DataLoader(test_dataset)
    
    return train_dataloader, dev_dataloader, test_dataloader

In [22]:
t_dataset = np.load('./input/dataset.npy')
t_out = np.load('./input/nor_out.npy')

print(t_dataset.shape)
print(t_out.shape)

train_dataloader, dev_dataloader, test_dataloader = data_dataloader(t_dataset, t_out, train_proportion=0.95, dev_proportion=0.025)

(4000, 3, 33, 49)
(4000, 5)
train_index : 3800
dev_index : 3900
train_data.shape : (3800, 3, 33, 49)
(3800, 5)
(100, 3, 33, 49)
(100, 5)
(100, 3, 33, 49)
(100, 5)
torch.Size([100, 3, 33, 49])
torch.Size([100, 5])


In [ ]:
'''
in the paper : 49 layers, 33 input, 18838 parameters
input : 10-weights(*input), 6 - biases
Y: 1 weight(hidden*output), 1 bias(output)
Input : hidden : output : layer  = # of parameters : len(para)
1:1:1:1 = 18 : 18
2:1:1:1 = 25 : 18  // +7 as expected
1:1:1:2 = 34 : 18 // 34 = 16*2 + 2
33:33:1:1 = 562 : 18 // 16*33(528) + 33*1 +1 = 562
33:33:5:1 = 698 : 18 // 16*33(528) + 33*5(165) +5 = 698
33:33:5:49 = 26042 : 18 // 16*33*49(25872) + 33*5(165) +5 = 698
weights = 10*33*49(16170) + 33*5(165) = 16335 gap : 2503

'''

In [35]:
input_size = 33
hidden_size = 33
output_size = 5
num_layers = 49
num_epochs = 1

x_mean = torch.Tensor(np.load('./input/x_nor_mean.npy'))

model = GRUD(input_size = input_size, hidden_size= hidden_size, output_size= output_size, x_mean=x_mean, num_layers=num_layers)

count = count_parameters(model)
print('number of parameters : ' , count)

GRUD is ready
number of parameters :  26042


In [36]:

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for t in range(num_epochs):
    for train_data, train_label in train_dataloader:
        # Squeeze the data [1, 33, 49], [1,5] to [33, 49], [5]
        train_data = torch.squeeze(train_data)
        train_label = torch.squeeze(train_label)
        
        # Forward pass : Compute predicted y by passing train data to the model
        y_pred = model(train_data)

        # Compute and print loss
        loss = criterion(y_pred, train_label)
        print(t, loss.item())
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
torch.Size([33])
tensor([0.8616, 1.0000, 1.0000, 1.0000, 1.0000, 0.8998, 1.0000, 0.9764, 1.0000,
        0.9949, 1.0000, 1.0000, 1.0000, 0.8829, 1.0000, 0.8993, 0.9079, 1.0000,
        1.0000, 0.9744, 1.0000, 0.9344, 0.9033, 0.9631, 0.9972, 1.0000, 0.9788,
        1.0000, 1.0000, 0.9640, 1.0000, 0.8341, 0.8559],
       grad_fn=<ExpBackward>)
torch.Size([33])
tensor([0.0530, 0.0344, 0.0275, 0.5514, 0.1392, 0.0610, 0.4743, 0.0681, 0.2212,
        0.5452, 0.7599, 0.1238, 0.4623, 0.4964, 0.2917, 0.1806, 0.0998, 0.2658,
        0.2048, 0.7857, 0.4047, 0.3008, 0.1822, 0.2013, 0.9664, 0.4024, 0.8790,
        0.1454, 0.0481, 0.0110, 0.0676, 0.2780, 0.0102])
torch.Size([33])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.])
torch.Size([33])
tensor([8.

C:\Users\woare\Anaconda3\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Users\woare\Anaconda3\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")



torch.Size([33])
tensor([0.9319, 0.9480, 0.9065, 1.0000, 1.0000, 0.8415, 0.9958, 0.9187, 1.0000,
        1.0000, 1.0000, 0.9104, 1.0000, 0.9097, 1.0000, 1.0000, 0.8955, 1.0000,
        0.8949, 1.0000, 1.0000, 0.9314, 1.0000, 0.7902, 0.9099, 0.8410, 0.0800,
        1.0000, 1.0000, 0.0103, 0.9671, 0.9689, 1.0000],
       grad_fn=<ExpBackward>)
torch.Size([33])
tensor([0.0530, 0.0344, 0.0275, 0.5514, 0.1392, 0.0610, 0.4743, 0.0681, 0.2212,
        0.5452, 0.7599, 0.1238, 0.4623, 0.4964, 0.2917, 0.1806, 0.0998, 0.2658,
        0.2048, 0.7857, 0.4047, 0.3008, 0.1822, 0.2013, 0.9664, 0.4024, 0.8790,
        0.1454, 0.0481, 0.0110, 0.0676, 0.2780, 0.0102])
torch.Size([33])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
torch.Size([33])
tensor([1.0000, 0.8645, 0.9728, 1.0000, 1.0000, 1.0000, 0.9200, 0.9923, 0.9908,
        0.9466, 1.0000, 1.0000, 0.9288, 0.9893, 0.0866, 1.0000, 0.9960, 0.9903

tensor([0.0530, 0.0344, 0.0275, 0.5514, 0.1392, 0.0610, 0.4743, 0.0681, 0.2212,
        0.5452, 0.7599, 0.1238, 0.4623, 0.4964, 0.2917, 0.1806, 0.0998, 0.2658,
        0.2048, 0.7857, 0.4047, 0.3008, 0.1822, 0.2013, 0.9664, 0.4024, 0.8790,
        0.1454, 0.0481, 0.0110, 0.0676, 0.2780, 0.0102])
torch.Size([33])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])
torch.Size([33])
tensor([1.0000e+00, 9.1588e-01, 1.0000e+00, 9.9531e-01, 9.0147e-01, 8.6068e-01,
        1.0000e+00, 1.0000e+00, 9.9470e-01, 1.0000e+00, 8.9727e-01, 8.4737e-01,
        9.8577e-01, 8.6644e-01, 1.0000e+00, 8.6265e-01, 1.0000e+00, 9.6705e-01,
        9.2668e-01, 1.0000e+00, 8.4921e-01, 1.0000e+00, 8.8298e-01, 1.0000e+00,
        8.6830e-01, 9.1542e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.5599e-09,
        9.3879e-01, 9.5469e-01, 9.7001e-01], grad_fn=<ExpBackward>)
torch.Size([33])
tensor([0.0530, 0.0344, 0.0275, 0.5

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
torch.Size([33])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 0.8656, 1.0000, 0.8615, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9536, 0.8532, 1.0000, 0.9099,
        1.0000, 0.9535, 1.0000, 1.0000, 1.0000, 0.1190, 1.0000, 1.0000, 0.9654,
        0.9910, 1.0000, 1.0000, 0.8594, 1.0000, 1.0000],
       grad_fn=<ExpBackward>)
torch.Size([33])
tensor([0.0530, 0.0344, 0.0275, 0.5514, 0.1392, 0.0610, 0.4743, 0.0681, 0.2212,
        0.5452, 0.7599, 0.1238, 0.4623, 0.4964, 0.2917, 0.1806, 0.0998, 0.2658,
        0.2048, 0.7857, 0.4047, 0.3008, 0.1822, 0.2013, 0.9664, 0.4024, 0.8790,
        0.1454, 0.0481, 0.0110, 0.0676, 0.2780, 0.0102])
torch.Size([33])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.])
torch.Size([33])
tensor([1.

torch.Size([33])
tensor([0.0530, 0.0344, 0.0275, 0.5514, 0.1392, 0.0610, 0.4743, 0.0681, 0.2212,
        0.5452, 0.7599, 0.1238, 0.4623, 0.4964, 0.2917, 0.1806, 0.0998, 0.2658,
        0.2048, 0.7857, 0.4047, 0.3008, 0.1822, 0.2013, 0.9664, 0.4024, 0.8790,
        0.1454, 0.0481, 0.0110, 0.0676, 0.2780, 0.0102])
torch.Size([33])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.])
torch.Size([33])
tensor([1.0000e+00, 1.0000e+00, 1.0000e+00, 8.6518e-01, 9.6821e-01, 1.0000e+00,
        9.4764e-01, 9.5574e-01, 6.1248e-04, 9.4190e-01, 9.3889e-01, 1.0000e+00,
        9.6818e-01, 9.3357e-01, 6.8780e-07, 9.6885e-01, 1.0000e+00, 2.1200e-02,
        9.4913e-01, 9.8817e-01, 1.0000e+00, 8.8378e-01, 8.7335e-01, 1.0000e+00,
        8.6607e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        8.9453e-01, 1.0000e+00, 1.0000e+00], grad_fn=<ExpBackward>)
torch.Size([33])
tensor([0.0530, 0.

torch.Size([33])
tensor([0.9319, 0.9480, 0.9065, 1.0000, 1.0000, 0.8415, 0.9958, 0.9187, 0.0010,
        1.0000, 0.9392, 0.9104, 1.0000, 0.9097, 1.0000, 1.0000, 0.8955, 1.0000,
        0.8949, 1.0000, 1.0000, 0.9314, 1.0000, 1.0000, 0.9099, 0.0079, 0.0780,
        1.0000, 1.0000, 0.9874, 0.9671, 0.9689, 1.0000],
       grad_fn=<ExpBackward>)
torch.Size([33])
tensor([0.0530, 0.0344, 0.0275, 0.5514, 0.1392, 0.0610, 0.4743, 0.0681, 0.2212,
        0.5452, 0.7599, 0.1238, 0.4623, 0.4964, 0.2917, 0.1806, 0.0998, 0.2658,
        0.2048, 0.7857, 0.4047, 0.3008, 0.1822, 0.2013, 0.9664, 0.4024, 0.8790,
        0.1454, 0.0481, 0.0110, 0.0676, 0.2780, 0.0102])
torch.Size([33])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0.])
torch.Size([33])
tensor([1.0000e+00, 8.6454e-01, 9.7280e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        9.2002e-01, 9.9230e-01, 7.2551e-05, 9.4659e-01, 1.0000e+00, 1.0000e+00,

tensor([0.0530, 0.0344, 0.0275, 0.5514, 0.1392, 0.0610, 0.4743, 0.0681, 0.2212,
        0.5452, 0.7599, 0.1238, 0.4623, 0.4964, 0.2917, 0.1806, 0.0998, 0.2658,
        0.2048, 0.7857, 0.4047, 0.3008, 0.1822, 0.2013, 0.9664, 0.4024, 0.8790,
        0.1454, 0.0481, 0.0110, 0.0676, 0.2780, 0.0102])
torch.Size([33])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.])
torch.Size([33])
tensor([8.7968e-01, 9.8168e-01, 9.2770e-01, 9.2362e-01, 9.6645e-01, 1.0000e+00,
        1.0000e+00, 9.8740e-01, 1.0000e+00, 9.1701e-01, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0713e-02,
        1.0000e+00, 9.2433e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00, 8.4461e-01,
        9.1437e-01, 1.8737e-05, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 8.7657e-01], grad_fn=<ExpBackward>)
torch.Size([33])
tensor([0.0530, 0.0344, 0.0275, 0.5

tensor([0.0530, 0.0344, 0.0275, 0.5514, 0.1392, 0.0610, 0.4743, 0.0681, 0.2212,
        0.5452, 0.7599, 0.1238, 0.4623, 0.4964, 0.2917, 0.1806, 0.0998, 0.2658,
        0.2048, 0.7857, 0.4047, 0.3008, 0.1822, 0.2013, 0.9664, 0.4024, 0.8790,
        0.1454, 0.0481, 0.0110, 0.0676, 0.2780, 0.0102])
torch.Size([33])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.])
torch.Size([33])
tensor([9.7081e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00, 9.8522e-01, 9.2460e-01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 8.5916e-01, 2.1830e-01, 1.0000e+00,
        9.5684e-01, 8.6181e-01, 1.0000e+00, 9.0073e-01, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 9.3294e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 2.2755e-05, 1.0000e+00, 1.0000e+00, 8.6561e-01, 8.9419e-01,
        1.0000e+00, 1.0000e+00, 9.5658e-01], grad_fn=<ExpBackward>)
torch.Size([33])
tensor([0.0530, 0.0344, 0.0275, 0.5

KeyboardInterrupt: 